In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
from datetime import datetime  
from datetime import timedelta
from multiprocessing import Manager,Process, Pool
import os
import numpy as np
import time
import re
from dateutil import tz

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}



In [2]:
def get_content(url,headers):    
    content = ''
    lista = []
    req = requests.get(url=url, headers=headers)
    if req.status_code == 200:
        print('Requisição bem sucedida na url: ',url)
        content = req.content
        lista.append(content)
        lista.append(url)
    return lista

In [3]:
def tabela_content(tabela):
    try:
        lista_all = []
        for i in range(len(tabela.contents)):
            if len(tabela.contents[i]) >1:
                if tabela.contents[i].find("td",{"class":"dt_n"}) is not None: #none o jogo está acontecendo
                    lista_aux = []
                    lista_aux.append(tabela.contents[i].find("td",{"class":"dt_n"})["data-dt"])
                    aux = tabela.contents[i].findAll("a")
                    for a in range(len(aux)):
                        lista_aux.append(aux[a].text.strip())
                    lista_aux.append('https://betsapi.com' + tabela.contents[i].find("a",href=re.compile("/r"))['href'])
                    lista_all.append(lista_aux)
                
        return lista_all
    except Exception as ex:
        print(ex)
        exit()

In [258]:
## PROCESSO PARA COLETAR OS RESULTADOS        
aux = 0
lista_fim = []
result_parcial = []
aux_arquivo_existe = 0
arquivo = 'ping_pong.xlsx'

url = 'https://betsapi.com/le/22534/TT-Cup-Ukraine/p.1'

for i in os.listdir('.'):
    if i == arquivo:
        df_temp = pd.read_excel(arquivo) #arquivo ja existente
        aux_arquivo_existe = 1
        
while aux == 0:
    
    time.sleep(1)
    content = get_content(url,headers)

    soup = BeautifulSoup(content[0], 'html.parser')

    paginacao = soup.findAll("div", {"class":"pagination-centered"})

    paginacao_ = soup.findAll("li", {"class":"page-item page-next"})

    next_page = ""
    if len(paginacao_) > 0:
        next_page = paginacao[0].find('li', {"class":"page-item page-next"}).find("a")["href"]
        url = 'https://betsapi.com'+next_page
        aux = 0
        if 'p.15' in next_page:
            aux = 1
        #else:
        #    url = 'https://pt.betsapi.com'+next_page
        #    aux = 0
    else:
        aux = 1
        
    pagina = soup.findAll("table", {"class": "table table-sm"})
    tabela = pagina[0].find("tbody")

    result_parcial = tabela_content(tabela)
    lista_fim.extend(result_parcial)

colunas = ['Data/Hora','Player1','Player2','Placar','URL_game']
df = pd.DataFrame(lista_fim, columns=colunas)



## coluna confrontos players unicos
df["id"] = df.index + 1

temp = df[['Player1','Player2']].copy()
x = pd.DataFrame(np.sort(temp, axis=1), temp.index, temp.columns)
x['id'] = x.index + 1
x['confronto'] = x['Player1'] + ' x ' + x['Player2']

df = df.merge(x[['id','confronto']], left_on='id', right_on='id')

df.drop('id',axis=1,inplace=True)
df_fim = df

df_fim['Data/Hora'] = df_fim['Data/Hora'].apply(lambda x : datetime.strptime(x, '%Y-%m-%dT%H:%M:%SZ'))
df_fim['Data/Hora'] = df_fim['Data/Hora'].apply(lambda x : x - timedelta(hours= 3) )
    
if aux_arquivo_existe == 1:
    frames = [df_fim, df_temp]

    result = pd.concat(frames)
    result.drop_duplicates(inplace=True)
    
else:
    result = df_fim
    
result.drop('id_jogo',axis=1,inplace=True)
result.reset_index(drop = True,inplace = True)
result.drop_duplicates(inplace = True)

result['id_jogo'] = result.URL_game.apply(lambda x : x.split('/r/')[1].split('/')[0])
result['id_jogo'] = result.id_jogo.apply(lambda x : x.strip())

Requisição bem sucedida na url:  https://betsapi.com/le/22534/TT-Cup-Ukraine/p.1
Requisição bem sucedida na url:  https://betsapi.com/le/22534/TT-Cup-Ukraine/p.2
Requisição bem sucedida na url:  https://betsapi.com/le/22534/TT-Cup-Ukraine/p.3
Requisição bem sucedida na url:  https://betsapi.com/le/22534/TT-Cup-Ukraine/p.4
Requisição bem sucedida na url:  https://betsapi.com/le/22534/TT-Cup-Ukraine/p.5
Requisição bem sucedida na url:  https://betsapi.com/le/22534/TT-Cup-Ukraine/p.6
Requisição bem sucedida na url:  https://betsapi.com/le/22534/TT-Cup-Ukraine/p.7
Requisição bem sucedida na url:  https://betsapi.com/le/22534/TT-Cup-Ukraine/p.8
Requisição bem sucedida na url:  https://betsapi.com/le/22534/TT-Cup-Ukraine/p.9
Requisição bem sucedida na url:  https://betsapi.com/le/22534/TT-Cup-Ukraine/p.10
Requisição bem sucedida na url:  https://betsapi.com/le/22534/TT-Cup-Ukraine/p.11
Requisição bem sucedida na url:  https://betsapi.com/le/22534/TT-Cup-Ukraine/p.12
Requisição bem sucedida n

AttributeError: 'DataFrame' object has no attribute 'id_jogo'

In [17]:
df_url = pd.read_excel('ping_pong.xlsx')
#df_url.head()

,Data/Hora,Player1,Player2,Placar,URL_game,id_jogo,confronto
0,2020-10-13 14:20:00,Viktoriia Pryshchepa,Ulyana Palamarchuk,2-3,https://betsapi.com/r/2841186/Viktoriia-Pryshc...,2841186,Ulyana Palamarchuk x Viktoriia Pryshchepa
1,2020-10-13 14:15:00,Maksim Samaray,Mykola Lytvynenko,1-3,https://betsapi.com/r/2838638/Maksim-Samaray-v...,2838638,Maksim Samaray x Mykola Lytvynenko
2,2020-10-13 14:10:00,Artem Vakulin,Anton Riabukhin,0-3,https://betsapi.com/r/2841043/Artem-Vakulin-v-...,2841043,Anton Riabukhin x Artem Vakulin
3,2020-10-13 14:05:00,Denis Kebalo,Ruslan Ivanov,2-3,https://betsapi.com/r/2841036/Denis-Kebalo-v-R...,2841036,Denis Kebalo x Ruslan Ivanov
4,2020-10-13 13:55:00,Vasyl Verbianyi,Nazarii Kovalchuk,2-3,https://betsapi.com/r/2840653/Vasyl-Verbianyi-...,2840653,Nazarii Kovalchuk x Vasyl Verbianyi


In [19]:
df_url = pd.read_excel('ping_pong.xlsx')
jogos = df_url.URL_game
for i in range(len(jogos)):
    print(jogos[i])

https://betsapi.com/r/2841186/Viktoriia-Pryshchepa-v-Ulyana-Palamarchuk
https://betsapi.com/r/2838638/Maksim-Samaray-v-Mykola-Lytvynenko
https://betsapi.com/r/2841043/Artem-Vakulin-v-Anton-Riabukhin
https://betsapi.com/r/2841036/Denis-Kebalo-v-Ruslan-Ivanov
https://betsapi.com/r/2840653/Vasyl-Verbianyi-v-Nazarii-Kovalchuk
https://betsapi.com/r/2840735/Nazarii-Hrytsyienko-v-Andrii-Kotik
https://betsapi.com/r/2841185/Iryna-Aseieva-v-Nataliia-Khmilevska
https://betsapi.com/r/2840598/Mykola-Lytvynenko-v-Nazarii-Mosiuk
https://betsapi.com/r/2841315/Sergey-Demidov-v-Oleg-Obidnyi
https://betsapi.com/r/2841519/Nodar-Kukava-v-Andrey-Golnik
https://betsapi.com/r/2841314/Kukava-Nodar-v-Andrey-Golnik
https://betsapi.com/r/2840652/Viktor-Didukh-v-Vasyl-Kushnir
https://betsapi.com/r/2840742/Andrii-Kotik-v-Vadym-Nechyporuk
https://betsapi.com/r/2841184/Nataliia-Khmilevska-v-Ulyana-Palamarchuk
https://betsapi.com/r/2841264/Maksim-Samaray-v-Levko-Kuzyo
https://betsapi.com/r/2838637/Maksim-Samaray-v-Lev

https://betsapi.com/r/2757900/Oleksandr-Tkachenko-v-Maxim-Nikolenko
https://betsapi.com/r/2759180/Artem-Derevianko-v-Oleksandr-Hrebelnyi
https://betsapi.com/r/2759165/Serhii-Tsybulin-v-Mykola-Telnoi
https://betsapi.com/r/2759357/Rodion-Pirveli-v-Levko-Kuzyo
https://betsapi.com/r/2759195/Sergey-Filatov-v-Mykola-Lytvynenko
https://betsapi.com/r/2757982/Rodion-Pirveli-v-Lev-Kuzyo
https://betsapi.com/r/2759345/Valeriy-Tatarinov-v-Dmytro-Zeniuk
https://betsapi.com/r/2757897/Valerii-Tatarynov-v-Dmytro-Zeniuk
https://betsapi.com/r/2757980/Dmytro-Baistriuchenko-v-Vladimir-Savoliuk
https://betsapi.com/r/2757885/Denys-Scherbak-v-Roman-Derypaska
https://betsapi.com/r/2759178/Vladyslav-Volokhov-v-Kyrylo-Yehorov
https://betsapi.com/r/2759159/Viktor-Didukh-v-Yurii-Teteria
https://betsapi.com/r/2759155/Rodion-Pirveli-v-Mikhail-Popov
https://betsapi.com/r/2757972/Rodion-Pirveli-v-Mykhailo-Popov
https://betsapi.com/r/2754664/Oleksandr-Zhukovskyi-v-Mykola-Lytvynenko
https://betsapi.com/r/2757912/Vitalii

https://betsapi.com/r/2724298/Artem-Vadanuk-v-Stanislav-Stelmashchuk
https://betsapi.com/r/2724134/Vitali-Rychkou-v-Roman-Rybachuk
https://betsapi.com/r/2726618/Evgeniya-Brekhova-v-Hanna-Helshtein
https://betsapi.com/r/2721524/Vadim-Tokar-v-Andrii-Shcherinskyi
https://betsapi.com/r/2725597/Sergey-Kinash-v-Vladyslav-Ivanchatenko
https://betsapi.com/r/2725596/Mykola-Zakladnyi-v-Aleksander-Dzyabura
https://betsapi.com/r/2724297/Yaroslav-Vorsuliak-v-Volodymyr-Ivasiv
https://betsapi.com/r/2724133/Roman-Rybachuk-v-Serhii-Berezynskyi
https://betsapi.com/r/2725560/Olha-Yehorova-v-Kateryna-Alekseenko
https://betsapi.com/r/2721523/Maksim-Samaray-v-Dmytro-Alekseenko
https://betsapi.com/r/2724149/Serhii-Kinash-v-Vladyslav-Ivanchatenko
https://betsapi.com/r/2724148/Oleksandr-Semenets-v-Oleksandr-Dziabura
https://betsapi.com/r/2724296/Volodymyr-Ivasiv-v-Vasyl-Aksenin
https://betsapi.com/r/2724132/Vitali-Rychkou-v-Sergii-Lugovskyi
https://betsapi.com/r/2725299/Kateryna-Alekseenko-v-Polina-Merzlikina


https://betsapi.com/r/2694946/Oleksandr-Pinchuk-v-Vasil-Salivonchyk
https://betsapi.com/r/2694963/Volodymyr-Maksiuta-v-Andrii-Hordii
https://betsapi.com/r/2694945/Andrey-Matcyk-v-Maxim-Naumchuk
https://betsapi.com/r/2694962/Oleksii-Haponov-v-Oleksandr-Shkurupii
https://betsapi.com/r/2694944/Vasil-Salivonchyk-v-Oleksandr-Mykhalus
https://betsapi.com/r/2694961/Andrii-Hordii-v-Volodymyr-Plishylo
https://betsapi.com/r/2694943/Oleksandr-Pinchuk-v-Maxim-Naumchuk
https://betsapi.com/r/2694960/Volodymyr-Maksiuta-v-Oleksandr-Shkurupii
https://betsapi.com/r/2697263/Andriy-Chermenskyi-v-Oleksandr-Kovalov
https://betsapi.com/r/2697260/Valeriy-Tatarinov-v-Anton-Riabukhin
https://betsapi.com/r/2696712/Oleksandr-Kostiuk-v-Serhii-Hilitukha
https://betsapi.com/r/2694904/Anton-Riabukhin-v-Valeriy-Tatarinov
https://betsapi.com/r/2694903/Oleksandr-Kovalov-v-Andriy-Chermenskyi
https://betsapi.com/r/2696666/Stanislav-Stelmashchuk-v-Oleg-Savenkov
https://betsapi.com/r/2696711/Andrey-Tsikhotskiy-v-Volodymyr-T

https://betsapi.com/r/2823837/Andrey-Krysak-v-Valentin-Bondar
https://betsapi.com/r/2823523/Halyna-Telna-v-Viktoriia-Pryshchepa
https://betsapi.com/r/2823439/Andriy-Shershov-v-Vladyslav-Batenko
https://betsapi.com/r/2822388/Sergey-Rubtsov-v-Anatoliy-Reva
https://betsapi.com/r/2821609/Roman-Derypaska-v-Yevhen-Masko
https://betsapi.com/r/2823567/Nazarii-Haievyi-v-Vitalii-Zharskiy
https://betsapi.com/r/2823522/Viktoriia-Pryshchepa-v-Olena-Telezhinskaya
https://betsapi.com/r/2823445/Mykola-Ulianchych-v-Bogdan-Panchenko
https://betsapi.com/r/2822387/Konstantin-Makarchuk-v-Vadym-Paukov
https://betsapi.com/r/2821602/Nikita-Trofimov-v-Maxim-Nikolenko
https://betsapi.com/r/2824196/Ivan-Mai-v-Mykola-Telnoi
https://betsapi.com/r/2823090/Uriy-Sandulenko-v-Serhii-Rutskyi
https://betsapi.com/r/2824081/Stepan-Nykyforuk-v-Ivan-Synytskyi
https://betsapi.com/r/2824058/Mykola-Serediuk-v-Igor-Demchuk
https://betsapi.com/r/2824032/Serhii-Zazymko-v-Vasyl-Aksenin
https://betsapi.com/r/2824031/Serhii-Tsybulin

https://betsapi.com/r/2801859/Sergey-Filatov-v-Mikhail-Rikun
https://betsapi.com/r/2801633/Roman-Derypaska-v-Serhii-Tsybulin
https://betsapi.com/r/2801840/Liubomyr-Sheshurak-v-Vitalii-Zharskiy
https://betsapi.com/r/2801879/Viktor-Didukh-v-Vadym-Nechyporuk
https://betsapi.com/r/2803518/Kateryna-Hubanova-v-Lilia-Melnikova
https://betsapi.com/r/2801662/Dmytro-Drobov-v-Yurii-Teteria
https://betsapi.com/r/2801794/Oleksandr-Dziabura-v-Vadym-Komar
https://betsapi.com/r/2801819/Bohdan-Sinkevych-v-Bohdan-Kohut
https://betsapi.com/r/2801858/Mikhail-Rikun-v-Mykola-Lytvynenko
https://betsapi.com/r/2801645/Serhii-Tsybulin-v-Nikita-Trofimov
https://betsapi.com/r/2801839/Vitalii-Zharskiy-v-Nazarii-Haievyi
https://betsapi.com/r/2801878/Vadym-Nechyporuk-v-Dmytro-Vakhta
https://betsapi.com/r/2803395/Lilia-Melnikova-v-Anastasiia-Postol
https://betsapi.com/r/2801661/Yurii-Teteria-v-Oleksandr-Tkachenko
https://betsapi.com/r/2801793/Mykola-Zakladnyi-v-Pavlo-Cherednichenko
https://betsapi.com/r/2801822/Alexa

In [5]:
url_jogo = df.loc[1]['URL_game']

content_jogo = get_content(url_jogo ,headers)

soup_jogo = BeautifulSoup(content_jogo[0], 'html.parser')

pagina_tabela_set_pontos = soup_jogo.findAll("div", {"class": "col-md-8"})[0].find("table")

Requisição bem sucedida na url:  https://betsapi.com/r/2838638/Maksim-Samaray-v-Mykola-Lytvynenko


In [36]:
jogos[jogos == 'https://betsapi.com/r/2836598/Oleksandr-Levadnyi-v-Nikita-Trofimov']

48    https://betsapi.com/r/2836598/Oleksandr-Levadn...
Name: URL_game, dtype: object

In [30]:
df_url = pd.read_excel('ping_pong.xlsx')
jogos = df_url.URL_game

list_pontos_set = []

for i in range(len(jogos)):
    time.sleep(2)
    url_jogo = jogos[i]

    content_jogo = get_content(url_jogo ,headers)

    soup_jogo = BeautifulSoup(content_jogo[0], 'html.parser')

    pagina_tabela_set_pontos = soup_jogo.findAll("div", {"class": "col-md-8"})[0].find("table")

    ## Bloco player 1

    list_pontos_aux = []

    list_pontos_aux.append(url_jogo) #Vai ser utilizada para fazer o join no primeiro DF
    for i in range(len(pagina_tabela_set_pontos.findAll("tr")[1].findAll('td'))):
        list_pontos_aux.append(pagina_tabela_set_pontos.findAll("tr")[1].findAll('td')[i].text)

    ## Bloco player 2
    list_pontos_set.append(list_pontos_aux)
    list_pontos_aux = []
    list_pontos_aux.append(url_jogo) #Vai ser utilizada para fazer o join no primeiro DF
    for i in range(len(pagina_tabela_set_pontos.findAll("tr")[2].findAll('td'))):
        list_pontos_aux.append(pagina_tabela_set_pontos.findAll("tr")[2].findAll('td')[i].text)

    list_pontos_set.append(list_pontos_aux)
    list_pontos_aux = []

Requisição bem sucedida na url:  https://betsapi.com/r/2841186/Viktoriia-Pryshchepa-v-Ulyana-Palamarchuk
Requisição bem sucedida na url:  https://betsapi.com/r/2838638/Maksim-Samaray-v-Mykola-Lytvynenko
Requisição bem sucedida na url:  https://betsapi.com/r/2841043/Artem-Vakulin-v-Anton-Riabukhin
Requisição bem sucedida na url:  https://betsapi.com/r/2841036/Denis-Kebalo-v-Ruslan-Ivanov
Requisição bem sucedida na url:  https://betsapi.com/r/2840653/Vasyl-Verbianyi-v-Nazarii-Kovalchuk
Requisição bem sucedida na url:  https://betsapi.com/r/2840735/Nazarii-Hrytsyienko-v-Andrii-Kotik
Requisição bem sucedida na url:  https://betsapi.com/r/2841185/Iryna-Aseieva-v-Nataliia-Khmilevska
Requisição bem sucedida na url:  https://betsapi.com/r/2840598/Mykola-Lytvynenko-v-Nazarii-Mosiuk
Requisição bem sucedida na url:  https://betsapi.com/r/2841315/Sergey-Demidov-v-Oleg-Obidnyi
Requisição bem sucedida na url:  https://betsapi.com/r/2841519/Nodar-Kukava-v-Andrey-Golnik
Requisição bem sucedida na url:

IndexError: list index out of range

In [32]:
df_jogo = pd.DataFrame(list_pontos_set)
df_jogo['id_jogo'] = df_jogo[0].apply(lambda x : x.split('/r/')[1].split('/')[0])
df_jogo.drop(0,inplace=True, axis = 1)
df_jogo.rename({1:'Player'},inplace=True, axis =1)

df_jogo_pivot = df_jogo.set_index(['id_jogo','Player']).stack().reset_index()
df_jogo_pivot.rename({0:'Pontos'},inplace=True, axis =1)
df_jogo_pivot.rename({'level_2':'Set'},inplace=True, axis =1)
df_jogo_pivot['Set'] = df_jogo_pivot['Set'].apply( lambda x : int(x)-1)

df_jogo_pivot

,id_jogo,Player,Set,Pontos
0,2841186,Viktoriia Pryshchepa,1,7
1,2841186,Viktoriia Pryshchepa,2,11
2,2841186,Viktoriia Pryshchepa,3,11
3,2841186,Viktoriia Pryshchepa,4,7
4,2841186,Viktoriia Pryshchepa,5,3
...,...,...,...,...
381,2836598,Oleksandr Levadniy,2,11
382,2836598,Oleksandr Levadniy,3,11
383,2836598,Nikita Trofimov,1,9
384,2836598,Nikita Trofimov,2,6


In [31]:
list_pontos_set

[['https://betsapi.com/r/2841186/Viktoriia-Pryshchepa-v-Ulyana-Palamarchuk',
  'Viktoriia Pryshchepa',
  '7',
  '11',
  '11',
  '7',
  '3'],
 ['https://betsapi.com/r/2841186/Viktoriia-Pryshchepa-v-Ulyana-Palamarchuk',
  'Ulyana Palamarchuk',
  '11',
  '9',
  '8',
  '11',
  '11'],
 ['https://betsapi.com/r/2838638/Maksim-Samaray-v-Mykola-Lytvynenko',
  'Maksim Samaray',
  '4',
  '6',
  '12',
  '4'],
 ['https://betsapi.com/r/2838638/Maksim-Samaray-v-Mykola-Lytvynenko',
  'Mykola Lytvynenko',
  '11',
  '11',
  '10',
  '11'],
 ['https://betsapi.com/r/2841043/Artem-Vakulin-v-Anton-Riabukhin',
  'Artem Vakulin',
  '9',
  '4',
  '7'],
 ['https://betsapi.com/r/2841043/Artem-Vakulin-v-Anton-Riabukhin',
  'Anton Riabukhin',
  '11',
  '11',
  '11'],
 ['https://betsapi.com/r/2841036/Denis-Kebalo-v-Ruslan-Ivanov',
  'Denis Kebalo',
  '7',
  '11',
  '11',
  '8',
  '4'],
 ['https://betsapi.com/r/2841036/Denis-Kebalo-v-Ruslan-Ivanov',
  'Ruslan Ivanov',
  '11',
  '9',
  '8',
  '11',
  '11'],
 ['https:/

In [42]:

df_nazarii = df_url[df_url['confronto'].str.contains("Nazarii")]
df_nazarii[df_nazarii['confronto'].str.contains("Oleksii Tkac")]


,Data/Hora,Player1,Player2,Placar,URL_game,id_jogo,confronto
121,2020-09-28 09:15:00,Nazarii Sydorak,Oleksii Tkachuk,1-3,https://betsapi.com/r/2795478/Nazarii-Sydorak-...,2795478,Nazarii Sydorak x Oleksii Tkachuk
210,2020-09-28 02:50:00,Nazarii Sydorak,Oleksii Tkachuk,3-1,https://betsapi.com/r/2792637/Nazarii-Sydorak-...,2792637,Nazarii Sydorak x Oleksii Tkachuk
